In [1]:
from ngsolve import *
import netgen.gui
from ngsolve import Draw, Redraw # just for visualization

optfile ./ng.opt does not exist - using default values
togl-version : 2
OCC module loaded
loading ngsolve library
NGSolve-6.2.2305-2-gcd17f9b83
Using Lapack
Including sparse direct solver UMFPACK
Running parallel using 16 thread(s)


In [2]:
import matplotlib.pyplot as plt
import numpy as np

In [3]:
from netgen.csg import *

left  = Plane (Pnt(0,0,0), Vec(-1,0,0) ).bc("dirichlet")
right = Plane (Pnt(1,1,1), Vec( 1,0,0) ).bc("dirichlet")
front = Plane (Pnt(0,0,0), Vec(0,-1,0) ).bc("dirichlet")
back  = Plane (Pnt(1,1,1), Vec(0, 1,0) ).bc("dirichlet")
bot   = Plane (Pnt(0,0,0), Vec(0,0,-1) ).bc("bottom")
top   = Plane (Pnt(1,1,1), Vec(0,0, 1) ).bc("top")


cube = left * right * front * back * bot * top

In [4]:
geo = CSGeometry()
geo.Add (cube)

mesh = geo.GenerateMesh(maxh=0.5)
#Redraw()
mesh = Mesh(mesh)
Draw(mesh)

 Start Findpoints
 Analyze spec points
 Find edges
 Start Findpoints
 Analyze spec points
 Find edges
 Start Findpoints
 Analyze spec points
 Find edges
 Surface 1 / 6
 Optimize Surface
 Surface 2 / 6
 Optimize Surface
 Surface 3 / 6
 Optimize Surface
 Surface 4 / 6
 Optimize Surface
 Surface 5 / 6
 Optimize Surface
 Surface 6 / 6
 Optimize Surface
 Delaunay meshing
 start tetmeshing
 Success !
 Remove Illegal Elements
 Volume Optimization


In [5]:
# set order of method
orderP = 3
# adaptive parameter theta=1 corresponds to uniform ref.
theta = 0.25;
# set max dofs
maxDOFs = 1000000
nDOFvisualize = maxDOFs
# Preconditioner
precNAME = "multigrid"
# precision
precCG = 1e-8

In [6]:
# ## Example 1: Known solution
# ExampleName = "Example1"
# exactSolKnown = True
#
# f = sin(pi*x)*sin(pi*y)+2*pi*pi*z*z/2*sin(pi*x)*sin(pi*y)
# g1 = 0
# g2 = 0
# v0 = 0
# sigma10 = 0
# sigma20 = 0

## Example 2: Pulse-type
ExampleName = "Example2"
exactSolKnown = False
x0 = 0.2
y0 = 0.2
v0 = -exp(-200*((x-x0)*(x-x0)+(y-y0)*(y-y0)))
sigma10 = exp(-200*((x-x0)*(x-x0)+(y-y0)*(y-y0)))
sigma20 = exp(-200*((x-x0)*(x-x0)+(y-y0)*(y-y0)))
f = 0
g1 = 400*exp(-200*((x-x0-z)*(x-x0-z)+(y-y0-z)*(y-y0-z)))*(y-y0-z)
g2 = 400*exp(-200*((x-x0-z)*(x-x0-z)+(y-y0-z)*(y-y0-z)))*(x-x0-z)

## Example 3: Solution with jumps
# ExampleName = "Example3"
# exactSolKnown = False
#
# f = 0
# g1 = 0
# g2 = 0
# v0 = 1
# sigma10 = 0
# sigma20 = 0

In [7]:
# Set filename were basic information (number of dofs, error estimator) are storedd
fNAMEresults = ExampleName + "2D_orderP" + str(orderP) + "_maxDof" + str(maxDOFs) + "_theta" + str(theta)
fNAMEresultsCSV = fNAMEresults + ".csv"

In [8]:
# FEM Spaces
V = H1(mesh, order=orderP, dirichlet="dirichlet")
SIGMA = H1(mesh, order=orderP)

fesm = V*SIGMA*SIGMA

In [9]:
v,sigma1,sigma2 = fesm.TrialFunction()
w,tau1,tau2 = fesm.TestFunction()

In [10]:
# Define and assemble bilinear form
blf = BilinearForm(fesm)
blf+= ( (grad(v)[2]-grad(sigma1)[0]-grad(sigma2)[1])*(grad(w)[2]-grad(tau1)[0]-grad(tau2)[1])  )*dx
blf+= ( (grad(sigma1)[2]-grad(v)[0])*(grad(tau1)[2]-grad(w)[0])  )*dx
blf+= ( (grad(sigma2)[2]-grad(v)[1])*(grad(tau2)[2]-grad(w)[1])  )*dx

# add boundary terms
blf+= ( v*w + sigma1*tau1 + sigma2*tau2 )*ds("bottom")


# register preconditioner
prec = Preconditioner(blf, precNAME)


blf.Assemble()

In [11]:
# Define Linear form
rhs = LinearForm(fesm)
rhs+= (f*(grad(w)[2]-grad(tau1)[0]-grad(tau2)[1]) + g1*(grad(tau1)[2]-grad(w)[0])+ g2*(grad(tau2)[2]-grad(w)[1]))*dx
rhs+= ( v0*w + sigma10*tau1 + sigma20*tau2 )*ds("bottom")

rhs.Assemble()


In [12]:
# grid function
gfm = GridFunction(fesm)
gfv, gfsigma1, gfsigma2 = gfm.components

In [13]:
#Define boundary2elements mapping
def GetB2E():
    #start = time.time()
   
    f2eAUX = np.zeros(mesh.nface,dtype=int)
    
    for el in mesh.Elements(VOL):
        f2eAUX[el.faces[0].nr] = el.nr
        f2eAUX[el.faces[1].nr] = el.nr
        f2eAUX[el.faces[2].nr] = el.nr
        f2eAUX[el.faces[3].nr] = el.nr
    
    nrBND = 0;
    for el in mesh.Elements(BND):
        nrBND += 1
    
    b2e = np.zeros(nrBND,dtype=int)
    
    
    for el in mesh.Elements(BND):
        b2e[el.nr] = f2eAUX[el.faces[0].nr]
    
    
    #end = time.time()
    #print("    Computing took ", (end - start), " seconds")
    return b2e

In [14]:
# realization of doerfler marking
def MarkBulk(eta,theta):
    indicators_sorted=np.sort(eta.NumPy())
    indicators_sorted[::-1] = indicators_sorted
    idx=np.argsort(eta.NumPy())
    idx[::-1]=idx

    

    sumind = np.cumsum(indicators_sorted)

    
    tmp = sumind>=sumind[len(sumind)-1]*theta
    jdx = np.min(np.where(tmp==True))

    
    
    marked = idx[0:jdx+1]
    #print(np.arange(0,jdx+1))
    #print(marked)

    tmp = np.full(len(eta.NumPy()), False)
    tmp[marked] = True
    #print(tmp)
    return tmp

In [15]:
l = []    # l = list of estimated total error

def CalcError():

    # compute estimator:
    gfm.Update()
    gfv.Update()
    gfsigma1.Update()
    gfsigma2.Update()

    errVol = (grad(gfv)[2]-grad(gfsigma1)[0]-grad(gfsigma2)[1]-f)*(grad(gfv)[2]-grad(gfsigma1)[0]-grad(gfsigma2)[1]-f)
    errVol += (grad(gfsigma1)[2]-grad(gfv)[0]-g1)*(grad(gfsigma1)[2]-grad(gfv)[0]-g1)
    errVol += (grad(gfsigma2)[2]-grad(gfv)[1]-g2)*(grad(gfsigma2)[2]-grad(gfv)[1]-g2)
    eta2 = Integrate(errVol, mesh, VOL, element_wise=True)
    
    # boundary errors
    errBou2 = Integrate((gfv-v0)*(gfv-v0),mesh,BND,element_wise=True,definedon=mesh.Boundaries("bottom"))
    errBou2+= Integrate((gfsigma1-sigma10)*(gfsigma1-sigma10),mesh,BND,element_wise=True,definedon=mesh.Boundaries("bottom"))
    errBou2+= Integrate((gfsigma2-sigma20)*(gfsigma2-sigma20),mesh,BND,element_wise=True,definedon=mesh.Boundaries("bottom"))

    
    tmp = eta2.NumPy()
    b2e = GetB2E() # compute boundary to volume element relation
    tmp[b2e] = tmp[b2e] + errBou2 # update error estimator (this adds directly to eta2)

    if(theta<1):
        mesh.ngmesh.Elements3D().NumPy()["refine"] = MarkBulk(eta2,theta)

    l.append ((fesm.ndof, sqrt(sum(eta2))))
    print("ndof =", fesm.ndof, " ErrEst =", sqrt(sum(eta2)))

In [ ]:
level = 0
steps = []
while fesm.ndof < maxDOFs:
    if level > 0:
        mesh.Refine()
    fesm.Update()
    gfm.Update()


    with TaskManager():
        blf.Assemble()
        rhs.Assemble()

        # Conjugate gradient solver
        inv = CGSolver(blf.mat, prec.mat, maxsteps=1000, precision=precCG)

        gfm.vec.data = inv * rhs.vec

    steps.append ( (fesm.ndof, inv.GetSteps()) )
    
    level = level+1
    CalcError() # compute error and mark elements

    if(fesm.ndof<nDOFvisualize):
        Draw(gfv)
        Redraw (blocking=True)


0 0.776666
1 0.0450033
2 0.0174355
3 0.00925675
4 0.00469399
5 0.00248792
6 0.00132594
7 0.000679004
8 0.000289865
9 0.000170407
10 8.13834e-05
11 4.6575e-05
12 1.99058e-05
13 1.02896e-05
14 5.8994e-06
15 2.50765e-06
16 1.59043e-06
17 7.46637e-07
18 3.82352e-07
19 1.69057e-07
20 8.30337e-08
21 3.71944e-08
22 1.82919e-08
23 9.77143e-09
24 4.47176e-09
ndof = 1164  ErrEst = 1.4100261279945316
 Mesh bisection
 Bisection done
0 1.07181
1 0.0734541
2 0.0373876
3 0.0215033
4 0.0135949
5 0.00859709
6 0.00558653
7 0.00413064
8 0.00255738
9 0.00141511
10 0.000884237
11 0.000516531
12 0.0003187
13 0.000206835
14 0.000130363
15 8.18774e-05
16 4.99187e-05
17 3.11538e-05
18 2.2413e-05
19 1.26533e-05
20 7.65545e-06
21 5.13588e-06
22 3.11785e-06
23 1.74376e-06
24 1.12742e-06
25 7.14683e-07
26 4.48336e-07
27 2.91884e-07
28 1.85765e-07
29 1.1548e-07
30 7.51262e-08
31 5.01535e-08
32 2.77543e-08
33 1.79331e-08
34 1.17662e-08
35 7.1485e-09
ndof = 2748  ErrEst = 0.9829060985430864
 Mesh bisection
 Bisection

In [ ]:
plt.yscale('log')
plt.xscale('log')
plt.xlabel("ndof")
plt.ylabel("estimator")
ndof,errEst = zip(*l)
hp = np.power((np.divide(1,ndof)*ndof[-1]),orderP/3)
plt.plot(ndof,errEst, "-*",ndof,hp*errEst[-1])

plt.ion()
plt.show()

In [ ]:
## save data (estimator and error) to csv file, for postprocessing 
np.savetxt(fNAMEresultsCSV, l, delimiter=',', header="ndof,errEst", comments="")

In [ ]:
gfm.Update()
gfv.Update()
Draw(gfv)

In [ ]:
# # VTKOutput object
# vtk = VTKOutput(ma=mesh,
#                coefs=[gfv,gfsigma1,gfsigma2],
#                 names = ["v","sigma1","gfsigma2"],
#                 filename=fNAMEresults,
#                 subdivision=orderP-1)
# # Exporting the results:
# vtk.Do()
